In [ ]:
import requests
import json

# Routes data (you need to fill in the coordinates)
routes = [
    {"start": {"lat": 33.7629, "lng": -84.4227}, "end": {"lat": 32.018, "lng": -81.1965}},
    {"start": {"lat": 32.018, "lng": -81.1965}, "end": {"lat": 33.3655, "lng": -82.0734}},
    {"start": {"lat": 33.3655, "lng": -82.0734}, "end": {"lat": 33.9496, "lng": -83.3701}},
    {"start": {"lat": 33.9496, "lng": -83.3701}, "end": {"lat": 32.8088, "lng": -83.6942}},
    {"start": {"lat": 32.8088, "lng": -83.6942}, "end": {"lat": 33.7629, "lng": -84.4227}},
]

# Google API Key
api_key = *HIDDEN*

# Google Maps API endpoint
url = 'https://routes.googleapis.com/directions/v2:computeRoutes'

# List to store route objects
route_objects = []

# Iterate over routes and make requests
for route in routes:
    start_lat = route["start"]["lat"]
    start_lng = route["start"]["lng"]
    end_lat = route["end"]["lat"]
    end_lng = route["end"]["lng"]

    # Request payload
    payload = {
        "origin": {"location": {"latLng": {"latitude": start_lat, "longitude": start_lng}}},
        "destination": {"location": {"latLng": {"latitude": end_lat, "longitude": end_lng}}},
        "travelMode": "DRIVE",
        "routingPreference": "TRAFFIC_AWARE",
        "departureTime": "2024-05-17T19:00:00Z",
        "computeAlternativeRoutes": True,
        "routeModifiers": {
            "avoidTolls": False,
            "avoidHighways": False,
            "avoidFerries": False
        },
        "languageCode": "en-US",
        "units": "IMPERIAL"
    }

    # API request headers
    headers = {
        'Content-Type': 'application/json',
        'X-Goog-Api-Key': api_key,
        'X-Goog-FieldMask': 'routes.legs.steps.distanceMeters,routes.routeLabels,routes.legs.steps.startLocation.latLng,routes.legs.steps.endLocation.latLng,routes.polyline.encodedPolyline'
    }

    # Make the request
    try:
        response = requests.post(url, headers=headers, json=payload)
        response.raise_for_status()  # Raise an exception for HTTP errors
        route_object = response.json()
        route_objects.append(route_object)
    except requests.exceptions.HTTPError as e:
        print(f"HTTP error occurred: {e}")
    except json.JSONDecodeError as e:
        print(f"JSON decoding error occurred: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

# Print or further process the route_objects list
print(route_objects[0])

{'routes': [{'legs': [{'steps': [{'distanceMeters': 750, 'startLocation': {'latLng': {'latitude': 33.762336, 'longitude': -84.422941}}, 'endLocation': {'latLng': {'latitude': 33.7578069, 'longitude': -84.4175546}}}, {'distanceMeters': 1859, 'startLocation': {'latLng': {'latitude': 33.7578069, 'longitude': -84.4175546}}, 'endLocation': {'latLng': {'latitude': 33.7411033, 'longitude': -84.4177778}}}, {'distanceMeters': 1812, 'startLocation': {'latLng': {'latitude': 33.7411033, 'longitude': -84.4177778}}, 'endLocation': {'latLng': {'latitude': 33.7446154, 'longitude': -84.3992918}}}, {'distanceMeters': 5339, 'startLocation': {'latLng': {'latitude': 33.7446154, 'longitude': -84.3992918}}, 'endLocation': {'latLng': {'latitude': 33.7039782, 'longitude': -84.4040902}}}, {'distanceMeters': 1241, 'startLocation': {'latLng': {'latitude': 33.7039782, 'longitude': -84.4040902}}, 'endLocation': {'latLng': {'latitude': 33.6931429, 'longitude': -84.4036423}}}, {'distanceMeters': 104359, 'startLocatio

In [ ]:
import pandas as pd

chunk_size = 100000

filtered_chunks = []

for chunk in pd.read_csv("US_Accidents_March23.csv", chunksize=chunk_size):
    filtered_chunk = chunk[chunk['State'] == 'GA']
    filtered_chunks.append(filtered_chunk)


df_ga = pd.concat(filtered_chunks, ignore_index=True)

y = df_ga['Severity']
X = df_ga.iloc[:, 11:]
X.drop(columns=['State', 'Country', 'Timezone', 'Airport_Code'], inplace = True)
X['Severity'] = y

X['Weather_Timestamp'] = pd.to_datetime(X['Weather_Timestamp'])
X['day_of_week'] = X['Weather_Timestamp'].dt.day_name()

def is_weekday(day):
    weekdays = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]
    return day in weekdays

# Apply the function to the day_of_week column to create the Weekday column
X['Weekday'] = X['day_of_week'].apply(lambda x: is_weekday(x))

X.drop(columns=['Weather_Timestamp', 'day_of_week'], inplace = True)

In [ ]:


# Calculate the average severity for accidents in each city
df = X.groupby('City')['Severity'].mean()
df = df.reset_index()

print(df)

             City  Severity
0         Acworth  2.505540
1     Adairsville  2.505660
2            Adel  2.612245
3          Adrian  2.000000
4           Ailey  3.052632
..            ...       ...
520          Wray  3.285714
521         Wrens  3.000000
522  Wrightsville  2.875000
523  Young Harris  2.666667
524       Zebulon  2.547170

[525 rows x 2 columns]


In [ ]:
pip install geopy



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from geopy.geocoders import Nominatim

In [ ]:
from geopy.exc import GeocoderTimedOut

def get_lat_lon(city, retries=3):
    geolocator = Nominatim(user_agent="my_application")
    for _ in range(retries):
        try:
            location = geolocator.geocode(city, timeout=10)
            if location:
                return location.latitude, location.longitude
        except GeocoderTimedOut:
            continue
    return None, None


In [ ]:
# Assuming your DataFrame is named df
df['Latitude'] = 0.0  # Initialize latitude column
df['Longitude'] = 0.0  # Initialize longitude column

for index, row in df.iterrows():
    city = row['City']  # Assuming the city name is the index
    latitude, longitude = get_lat_lon(city)
    df.at[index, 'Latitude'] = latitude
    df.at[index, 'Longitude'] = longitude

# Print the DataFrame to check the result
print(df)


             City  Severity   Latitude   Longitude
0         Acworth  2.505540  34.065933  -84.676880
1     Adairsville  2.505660  34.368327  -84.934906
2            Adel  2.612245  41.617412  -94.018518
3          Adrian  2.000000  41.897515  -84.037305
4           Ailey  3.052632  32.187404  -82.565689
..            ...       ...        ...         ...
520          Wray  3.285714  40.075882 -102.223253
521         Wrens  3.000000  33.207646  -82.391792
522  Wrightsville  2.875000  32.729328  -82.719859
523  Young Harris  2.666667  34.933145  -83.847122
524       Zebulon  2.547170  33.102345  -84.342700

[525 rows x 4 columns]


In [ ]:
print(df.loc[[df['Severity'].idxmax()]])

     City  Severity   Latitude  Longitude
31  Avera       4.0  43.389623  -0.176828


In [ ]:
print(df.loc[[df['Severity'].idxmin()]])

          City  Severity  Latitude  Longitude
203  Gracewood       1.0  28.26174 -82.649404


In [ ]:
import math

# Assuming df is the DataFrame containing city locations and severities
city_data = df.set_index('City').to_dict(orient='index')

# New list to store the results
result_dicts = []

# Iterate through route_objects
for route_obj in route_objects:
    # New dictionary to store the results
    result_dict = {}
    for route in route_obj['routes']:

        cumulative_danger = 0

        # Iterate through the legs in each route
        for leg in route['legs']:

            # Iterate through the steps in each leg
            for step in leg['steps']:
                start_lat = step['startLocation']['latLng']['latitude']
                start_lon = step['startLocation']['latLng']['longitude']
                end_lat = step['endLocation']['latLng']['latitude']
                end_lon = step['endLocation']['latLng']['longitude']

                # Calculate average location for the step
                avg_lat = (start_lat + end_lat) / 2
                avg_lon = (start_lon + end_lon) / 2

                # Find the corresponding city
                min_distance = float('inf')
                corresponding_city = None
                for city, city_info in city_data.items():
                    city_lat = city_info['Latitude']
                    city_lon = city_info['Longitude']
                    distance = math.acos(math.sin(math.radians(avg_lat)) * math.sin(math.radians(city_lat)) + math.cos(math.radians(avg_lat)) * math.cos(math.radians(city_lat)) * math.cos(math.radians(city_lon - avg_lon)))

                    if distance < min_distance:
                        min_distance = distance
                        corresponding_city = city

                # Pull the severity level
                severity_level = df.loc[df['City'] == corresponding_city, 'Severity'].values[0]

                # Multiply severity level by distance
                danger = severity_level * step['distanceMeters']

                print(severity_level, corresponding_city, danger)

                # Add to cumulative danger for the route
                cumulative_danger += danger

        result_dict[route['polyline']['encodedPolyline']]=cumulative_danger

    result_dicts.append(result_dict)

2.535845291545617 Atlanta 1901.8839686592128
2.535845291545617 Atlanta 4714.136396983302
2.535845291545617 Atlanta 4594.951668280658
2.535845291545617 Atlanta 13538.87801156205
2.0 East Point 2482.0
2.5036363636363634 Locust Grove 261276.98727272724
2.5888399412628487 Macon 51137.355359765046
2.5888399412628487 Macon 3386.202643171806
2.5888399412628487 Macon 2386.9104258443463
2.5888399412628487 Macon 652.3876651982379
2.5888399412628487 Macon 1382.4405286343613
2.4444444444444446 Swainsboro 635580.0
2.216834755624515 Savannah 7144.858417377812
2.216834755624515 Savannah 8607.969356089992
2.216834755624515 Savannah 678.3514352211016
2.216834755624515 Savannah 1190.4402637703645
2.216834755624515 Savannah 2323.2428238944917
2.535845291545617 Atlanta 1901.8839686592128
2.535845291545617 Atlanta 4714.136396983302
2.6554726368159205 Conyers 250543.8432835821
3.2 Newborn 1737.6000000000001
2.5301204819277108 Eatonton 155002.77108433735
2.5 Toomsboro 177240.0
2.74468085106383 East Dublin 22

In [ ]:
print(result_dicts)

[{'seamEjzgbO`A}EtAmGRq@`@}@v@w@rDaCjBkA`HoD`Ac@fDEvGBhFB`CBv@EjB?NAdN?`H?bJBxO?|KNTDnALbDFGc@EsAUiEEcCOQLyLR_Gb@wIDuDGeDOiC_@oDe@kCo@mC}@qCkLoZeAaDSs@D[c@qD_@sGAi@HSF_GLiCVyBXeB`@yAb@{@l@m@n@Wh@InA?fAAzABhF[hBEVUbYTbFJbKLvEFtCPbC^vDx@vHnBlDp@tCR|Mr@`BTlBf@tBx@dBdAjHfFnGlEpDvBzCtAzDlA|F|AzB|@lAj@lAx@xG`FvBnAhCfAhF|AxRjFbDp@hEn@dDXr@DzDLhIJdERdMzApDb@jBNzB@hBKtAWnA_@dBy@|ByAvAaAtCqBZYfV{P`BeA`By@hBo@hB_@jBQ~BE`JBfEI`EYva@sDhEQdEApX^xBCxBQvB]vF{AdV}GvFqA~Dk@rE]dCKdDCpCDfDNzBThEj@tB`@`JtB|@Xze@lLfJhCrFbBdPxD|EjAnEdAxDl@tDF|CClKu@v@AvGo@~NaAvESdQ{@nOiAxFo@fFy@|h@wJv[aGbEeAjE}AlEoBpb@iSlGiDtSoMfT_NlF_DxBeA`JaDfB}@zAcArBsBfA{AbAeBl@yAp@sBXiAr@eDjJqg@dDoQbAaFlA_Ht@uEfc@}fD|@eF`AiEpAwEfB_FdBaEjCeFzs@cnAfGeLxa@az@jBkEnAoDjAyDxAeGrTkaAxBmIj]akAbLw_@lIeYrBcFx@wAjAwAvAsAfHoF~NwKlDyChY{X|NcNbJyIl_@e_@~KyKv@{@`XwW|DiEbFiGng@as@|CyDrD}DrIaIdm@kj@pCcCbCkBnBsAzEmCxCqAtCaA|Cw@xL{Bl_@{GnCg@dZmFpUeE|HiBpIcC~d@sNxW}H~OiF|G}BrFaCbFiCdCyAxDiCtDwCdQyNle@c_@pOaLjLiJ|^mZfKkItDgDtEuExEkFlEwFv^}f@fAuAbBaC~t@gcAfL

In [ ]:
print(result_dicts[0].values())

dict_values([1002978.9962371802, 1161155.1915925636, 1121673.5862429407])


In [ ]:
print(result_dicts[0].keys())

dict_keys(['seamEjzgbO`A}EtAmGRq@`@}@v@w@rDaCjBkA`HoD`Ac@fDEvGBhFB`CBv@EjB?NAdN?`H?bJBxO?|KNTDnALbDFGc@EsAUiEEcCOQLyLR_Gb@wIDuDGeDOiC_@oDe@kCo@mC}@qCkLoZeAaDSs@D[c@qD_@sGAi@HSF_GLiCVyBXeB`@yAb@{@l@m@n@Wh@InA?fAAzABhF[hBEVUbYTbFJbKLvEFtCPbC^vDx@vHnBlDp@tCR|Mr@`BTlBf@tBx@dBdAjHfFnGlEpDvBzCtAzDlA|F|AzB|@lAj@lAx@xG`FvBnAhCfAhF|AxRjFbDp@hEn@dDXr@DzDLhIJdERdMzApDb@jBNzB@hBKtAWnA_@dBy@|ByAvAaAtCqBZYfV{P`BeA`By@hBo@hB_@jBQ~BE`JBfEI`EYva@sDhEQdEApX^xBCxBQvB]vF{AdV}GvFqA~Dk@rE]dCKdDCpCDfDNzBThEj@tB`@`JtB|@Xze@lLfJhCrFbBdPxD|EjAnEdAxDl@tDF|CClKu@v@AvGo@~NaAvESdQ{@nOiAxFo@fFy@|h@wJv[aGbEeAjE}AlEoBpb@iSlGiDtSoMfT_NlF_DxBeA`JaDfB}@zAcArBsBfA{AbAeBl@yAp@sBXiAr@eDjJqg@dDoQbAaFlA_Ht@uEfc@}fD|@eF`AiEpAwEfB_FdBaEjCeFzs@cnAfGeLxa@az@jBkEnAoDjAyDxAeGrTkaAxBmIj]akAbLw_@lIeYrBcFx@wAjAwAvAsAfHoF~NwKlDyChY{X|NcNbJyIl_@e_@~KyKv@{@`XwW|DiEbFiGng@as@|CyDrD}DrIaIdm@kj@pCcCbCkBnBsAzEmCxCqAtCaA|Cw@xL{Bl_@{GnCg@dZmFpUeE|HiBpIcC~d@sNxW}H~OiF|G}BrFaCbFiCdCyAxDiCtDwCdQyNle@c_@pOaLjLiJ|^mZfKkItDgDtEuExEkFlEwFv^}f@fAuAbBa

In [ ]:
# Coding Steps Remaining:

# Iterate through the steps in each route and calculate the average location for each step
# Compare to the latitude longitude locations for cities using an appropriate distance calculation to find the corresponding city
# Pull the corresponding severity level
# Multiply the severity level by the distance associated with the step
# Add all the products for each step
# Track the sums for each route
# Rank order them

# Notes:

# I can do all this pretty easily after the meeting if its helpful,
# the reason I wanted to check is because even if I do all that,
# there is the more serious challenge of using the created polylines and rankings to actually contribute something to the visualization,
# and if we don't think that's feasible we don't need to pursue it at all.

# Possibilities
# Python visualization, interactive with user
# Tableau like shown
# Focus on existing visualizations
# Checkpoint at 7PM

# Useful Packages
# https://gis.stackexchange.com/questions/421259/decoding-polyline-in-python
# https://python-visualization.github.io/folium/latest/user_guide/vector_layers/polyline.html

In [ ]:
import polyline
my_string = list(result_dicts[0].keys())[0]
l = polyline.decode(my_string)
print(l[:5])
print(len(l))

[(33.76234, -84.42294), (33.76201, -84.42183), (33.76158, -84.42048), (33.76148, -84.42023), (33.76131, -84.41992)]
1063


In [ ]:
import folium

# Separate latitude and longitude values into two lists
latitudes, longitudes = zip(*l)

# Calculate the mean of latitudes and longitudes
mean_latitude = sum(latitudes) / len(latitudes)
mean_longitude = sum(longitudes) / len(longitudes)

# Create the map and add the line
m = folium.Map(location=[mean_latitude, mean_longitude], zoom_start=8)

folium.PolyLine(
    locations=l,
    color="#FF0000",
    weight=5,
    tooltip="From Boston to San Francisco",
).add_to(m)

m

In [ ]:
latitudes = []
longitudes = []

# Choose element from list of options to plot
i = 0

for key in result_dicts[i].keys():
    l = polyline.decode(key)
    latitudes_temp, longitudes_temp = zip(*l)
    latitudes.extend(latitudes_temp)
    longitudes.extend(longitudes_temp)

# Calculate the mean of latitudes and longitudes
mean_latitude = sum(latitudes) / len(latitudes)
mean_longitude = sum(longitudes) / len(longitudes)

# Find the key associated with the highest value
best_key = min(result_dicts[i], key=result_dicts[i].get)

# Create the map and add the line
m = folium.Map(location=[mean_latitude, mean_longitude], zoom_start=8)

# Plot the polylines (excluding the best route)
for key in result_dicts[i].keys():
    if key != best_key:
        l = polyline.decode(key)
        color = "#FF0000"
        tooltip = "Alternative Route"
        folium.PolyLine(locations=l, color=color, weight=5, tooltip=tooltip).add_to(m)

# Plot the best route last
l = polyline.decode(best_key)
color = "#0000FF"
tooltip = "Best Route"
folium.PolyLine(locations=l, color=color, weight=5, tooltip=tooltip).add_to(m)

# Create legend
legend_html = '''
     <div style="position: fixed;
                 bottom: 50px; left: 50px; width: 180px; height: 90px;
                 border:2px solid grey; z-index:9999; font-size:14px;
                 background-color:white;
                 ">&nbsp; <b>Legend</b> <br>
                   &nbsp; Best Route &nbsp; <i class="fa fa-map-marker fa-2x" style="color:blue"></i><br>
                   &nbsp; Alternative Route &nbsp; <i class="fa fa-map-marker fa-2x" style="color:red"></i>
     </div>
     '''

m.get_root().html.add_child(folium.Element(legend_html))

m